## Imports

In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from catboost import CatBoostClassifier

## Loading data

In [3]:
PATH_TO_DATA = '../data/для анализа/'

In [4]:
df_normal = pd.read_csv(PATH_TO_DATA + 'df_normal_clean.csv')
df_anomaly = pd.read_csv(PATH_TO_DATA + 'df_anomaly_clean.csv')
df_problems = pd.read_csv(PATH_TO_DATA + 'df_problems_clean.csv')

/var/folders/_h/72vw5ztx7sz2jdjfzdkp77l80000gn/T/ipykernel_73835/660808868.py:1: DtypeWarning: Columns (1,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_normal = pd.read_csv(PATH_TO_DATA + 'df_normal_clean.csv')
/var/folders/_h/72vw5ztx7sz2jdjfzdkp77l80000gn/T/ipykernel_73835/660808868.py:2: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_anomaly = pd.read_csv(PATH_TO_DATA + 'df_anomaly_clean.csv')


## Generating working data

In [5]:
# Добавляем новый признак (является ли наблюдение аномалией)

df_anomaly['is_anomaly'] = 1
df_normal['is_anomaly'] = 0
df_problems['is_anomaly'] = 0

In [6]:
# Объединяем датасеты
df_concat = pd.concat([df_anomaly, df_normal, df_problems], axis=0)

In [7]:
df_concat.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [8]:
# Категориальные признаки
cat_features = [
    'Сост.пед.сцепл.',
]

# Бинарные признаки
bin_features = [
    'Нейтраль КПП (spn3843)',
    'Стояночный тормоз (spn3842)',
    'Аварийная температура охлаждающей жидкости (spn3841)',
    'Засоренность воздушного фильтра (spn3840)',
    'Засоренность фильтра КПП (spn3847)',
    'Аварийное давление масла ДВС (spn3846)',
    'Засоренность фильтра ДВС (spn3845)',
    'Засоренность фильтра рулевого управления (spn3844)',
    'Засоренность фильтра навесного оборудования (spn3851)',
    'Недопустимый уровень масла в гидробаке (spn3850)',
    'Аварийная температура масла в гидросистеме (spn3849)',
    'Аварийное давление в I контуре тормозной системы (spn3848)',
    'Аварийное давление в II контуре тормозной системы (spn3855)',
    'Зарядка АКБ (spn3854)',
    'Отопитель (spn3853)',
    'Выход блока управления двигателем (spn3852)',
    'Включение тормозков (spn3859)',
    'Засоренность фильтра слива (spn3858)',
    'Аварийное давление масла КПП (spn3857)',
    'Аварийная температура масла ДВС(spn3856)',
    'Неисправность тормозной системы (spn3863)',
    'Термостарт (spn3862)',
    'Разрешение запуска двигателя (spn3861)',
    'Низкий уровень ОЖ (spn3860)',
    'Аварийная температура масла ГТР (spn3867)',
    'Необходимость сервисного обслуживания (spn3866)',
    'Подогрев топливного фильтра (spn3865)',
    'Вода в топливе (spn3864)',
    'Холодный старт (spn3871)',
]

# Числовые признаки
num_features = [
    'Значение счетчика моточасов, час:мин',
    'Давление в пневмостистеме (spn46), кПа',
    'ДВС. Температура охлаждающей жидкости',
    'ДВС. Давление смазки',
    'КПП. Давление масла в системе смазки',
    'Давл.масла двиг.,кПа',
    'ДВС. Частота вращения коленчатого вала',
    'Электросистема. Напряжение',
    'Уровень топлива % (spn96)',
    'Скорость',
    'КПП. Температура масла',
    'Обор.двиг.,об/мин',
    'Полож.пед.акселер.,%',
    'Месяц',
    'День недели',
    'Темп.масла двиг.,°С'
]

catboost_features = cat_features + bin_features

In [12]:
# Разделяем на признаки и таргет
X = df_concat.drop(columns=['is_anomaly'], axis=1)
y = df_concat['is_anomaly']

In [13]:
# Делим данные на тренировочную, тестовую и валидационную часть
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

## Building CatBoostClassifier

In [15]:
# Инициализируем классификатор и обучаем его на данных

classifier = CatBoostClassifier()

classifier.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),
    cat_features=catboost_features,
    early_stopping_rounds=10,
)

Learning rate set to 0.207331
0:	learn: 0.1121668	test: 0.1121122	best: 0.1121122 (0)	total: 1.53s	remaining: 25m 32s
1:	learn: 0.0157165	test: 0.0157125	best: 0.0157125 (1)	total: 3.12s	remaining: 25m 58s
2:	learn: 0.0029964	test: 0.0029971	best: 0.0029971 (2)	total: 4.57s	remaining: 25m 17s
3:	learn: 0.0007771	test: 0.0007779	best: 0.0007779 (3)	total: 5.97s	remaining: 24m 47s
4:	learn: 0.0002679	test: 0.0002684	best: 0.0002684 (4)	total: 7.36s	remaining: 24m 25s
5:	learn: 0.0001403	test: 0.0001406	best: 0.0001406 (5)	total: 8.8s	remaining: 24m 18s
6:	learn: 0.0000734	test: 0.0000737	best: 0.0000737 (6)	total: 10.1s	remaining: 23m 55s
7:	learn: 0.0000494	test: 0.0000496	best: 0.0000496 (7)	total: 11.5s	remaining: 23m 43s
8:	learn: 0.0000327	test: 0.0000328	best: 0.0000328 (8)	total: 12.7s	remaining: 23m 19s
9:	learn: 0.0000262	test: 0.0000263	best: 0.0000263 (9)	total: 14s	remaining: 23m 3s
10:	learn: 0.0000191	test: 0.0000192	best: 0.0000192 (10)	total: 15.2s	remaining: 22m 44s
11:	

## Assesing metrics

In [17]:
y_pred = classifier.predict(X_test)

precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred)

(1.0, 1.0, 1.0)

In [18]:
classifier.save_model('anomaly_classifier')